<a href="https://colab.research.google.com/github/mohamed-elmelegy/seq2seq-machine-translation/blob/master/EnlgishToSpanishMachineTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the required libraries

In [23]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from nltk.translate.bleu_score import corpus_bleu

# Reading the data
[Dataset](https://www.manythings.org/anki/spa-eng.zip)

In [2]:
!wget https://www.manythings.org/anki/spa-eng.zip

--2021-12-12 20:08:49--  https://www.manythings.org/anki/spa-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.186.54, 104.21.92.44, 2606:4700:3030::6815:5c2c, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.186.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5326759 (5.1M) [application/zip]
Saving to: ‘spa-eng.zip.3’

spa-eng.zip.3       100%[===================>]   5.08M  7.24MB/s    in 0.7s    

2021-12-12 20:08:50 (7.24 MB/s) - ‘spa-eng.zip.3’ saved [5326759/5326759]



In [3]:
# !unzip /content/spa-eng.zip

In [4]:
# Path to the data txt file on disk.
data_path = "/content/spa.txt"  

# open the file eng-spa.txt and read
lines = pd.read_table(data_path,  names =['source', 'target', 'comments'])

#printing sample data from lines
lines.count()

source      138416
target      138416
comments    138416
dtype: int64

# Data Cleaning

In [5]:
# convert source and target text to Lowercase 
lines.source=lines.source.apply(lambda x: x.lower())
lines.target=lines.target.apply(lambda x: x.lower())

# Remove quotes from source and target text
lines.source=lines.source.apply(lambda x: re.sub("'", '', x))
lines.target=lines.target.apply(lambda x: re.sub("'", '', x))

# create a set of all special characters
special_characters= set(string.punctuation)

# Remove all the special characters
lines.source = lines.source.apply(lambda x: ''.join(char1 for char1 in x if char1 not in special_characters))
lines.target = lines.target.apply(lambda x: ''.join(char1 for char1 in x if char1 not in special_characters))

# Remove digits from source and target sentences
num_digits= str.maketrans('','', digits)
lines.source=lines.source.apply(lambda x: x.translate(num_digits))
lines.target= lines.target.apply(lambda x: x.translate(num_digits))

# Remove extra spaces
lines.source=lines.source.apply(lambda x: x.strip())
lines.target=lines.target.apply(lambda x: x.strip())
lines.source=lines.source.apply(lambda x: re.sub(" +", " ", x))
lines.target=lines.target.apply(lambda x: re.sub(" +", " ", x))


In [6]:
# Add start and end tokens to target sequences
lines.target = lines.target.apply(lambda x : 'START_ '+ x + ' _END')
lines.sample(6)

,source,target,comments
74809,tom put his ipad on the table,START_ tom puso su ipad sobre la mesa _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
88606,which countries have you visited,START_ ¿qué países has visitado _END,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
57456,he plays golf every sunday,START_ él juega al golf todos los domingos _END,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
31373,we cant give up now,START_ no nos podemos dar por vencidos ahora _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
97867,itll cost at least a hundred pesos,START_ por lo menos costará cien pesos _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
122147,there were no railroads in japan at that time,START_ en japón no había ferrocarril en esa ép...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [7]:
# Find all the source and target words and sort them
# Vocabulary of Source language
all_source_words=set()
for source in lines.source:
    for word in source.split():
        if word not in all_source_words:
            all_source_words.add(word)# Vocabulary of Target 
all_target_words=set()
for target in lines.target:
    for word in target.split():
        if word not in all_target_words:
            all_target_words.add(word)
# sort all unique source and target words
source_words= sorted(list(all_source_words))
target_words=sorted(list(all_target_words))

In [8]:
#Find maximum sentence length in  the source and target data
source_length_list=[]
for l in lines.source:
    source_length_list.append(len(l.split(' ')))
max_source_length= max(source_length_list)
print(" Max length of the source sentence",max_source_length)

target_length_list=[]
for l in lines.target:
    target_length_list.append(len(l.split(' ')))
max_target_length= max(target_length_list)
print(" Max length of the target sentence",max_target_length)

 Max length of the source sentence 70
 Max length of the target sentence 70


In [9]:
# creating a word to index(word2idx) for source and target
source_word2idx= dict([(word, i+1) for i,word in enumerate(source_words)])
target_word2idx=dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
#creating a dictionary for index to word for source and target vocabulary
source_idx2word= dict([(i, word) for word, i in  source_word2idx.items()])
target_idx2word =dict([(i, word) for word, i in target_word2idx.items()])

In [11]:
# Shuffle the data
lines = shuffle(lines)
reduced_lines = lines.sample(3000, random_state=42)

In [12]:
# Train - Test Split
X, y = reduced_lines.source, reduced_lines.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((2700,), (300,))

In [13]:
# Input tokens for encoder
num_encoder_tokens=len(source_words)# Input tokens for decoder zero padded
num_decoder_tokens=len(target_words) +1

In [14]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_source_length),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_target_length),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_target_length, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):encoder_input_data[i, t] = source_word2idx[word] 
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_word2idx[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        # print(word)
                        decoder_target_data[i, t - 1, target_word2idx[word]] = 1.
                    
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

# Build Sequence2Sequence model

In [15]:
# setup basic parameters
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50
latent_dim=256

## Build the Encoder

In [16]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

## Build the Decoder

In [17]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## Define the model

In [18]:
# Define the model that takes encoder and decoder input 
# to output decoder_outputs
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

### Model Training

In [19]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [20]:
train_samples = len(X_train) # Total Training samples
val_samples = len(X_test)    # Total validation or test samples
batch_size = 128
epochs = 100

In [21]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/100
21/21 [==============================] - 33s 1s/step - loss: 0.7664 - acc: 0.1360 - val_loss: 0.6480 - val_acc: 0.1429
Epoch 2/100
21/21 [==============================] - 19s 882ms/step - loss: 0.5835 - acc: 0.1456 - val_loss: 0.6405 - val_acc: 0.1529
Epoch 3/100
21/21 [==============================] - 19s 911ms/step - loss: 0.5637 - acc: 0.1608 - val_loss: 0.6333 - val_acc: 0.1657
Epoch 4/100
21/21 [==============================] - 19s 907ms/step - loss: 0.5493 - acc: 0.1685 - val_loss: 0.6304 - val_acc: 0.1652
Epoch 5/100
21/21 [==============================] - 19s 911ms/step - loss: 0.5396 - acc: 0.1717 - val_loss: 0.6235 - val_acc: 0.1752
Epoch 6/100
21/21 [==============================] - 19s 909ms/step - loss: 0.5273 - acc: 0.1747 - val_loss: 0.6241 - val_acc: 0.1830
Epoch 7/100
21/21 [==============================] - 19s 910ms/step - loss: 0.5211 - acc: 0.1795 - val_loss: 0.6224 - val_acc: 0.1808
Epoch 8/100
21/21 [==============================] - 19s 910ms/st

In [24]:
model.save_weights('nmt_weights_100epochs.h5')

### Define the inference model

In [25]:
# Encode the input sequence to get the "Context vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_input = [decoder_state_input_h, decoder_state_input_c]

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_state_input)
decoder_states2 = [state_h2, state_c2]
# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_state_input,
    [decoder_outputs2] + decoder_states2)

In [26]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of 
    #target sequence with the start character.
    target_seq[0, 0] = target_word2idx['START_']
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word =target_idx2word[sampled_token_index]
        decoded_sentence += ' '+ sampled_word
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True
            # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        # Update states
        states_value = [h, c]
        return decoded_sentence

# Make Predictions

In [27]:
# Predict training set
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [35]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_train[k:k+1].values[0])
print('Actual Target Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Target Translation:', decoded_sentence)

Input Source sentence: why ask me wouldnt it be better to do it yourself
Actual Target Translation:  ¿por qué me pides a mí ¿no sería mejor si lo hicieras tú misma 
Predicted Target Translation:  almorcé


In [36]:
# predict testing set
test_gen = generate_batch(X_test, y_test, batch_size = 1)
k=10
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_test[k:k+1].values[0])
print('Actual Target Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Target Translation:', decoded_sentence)

Input Source sentence: he is very depressed
Actual Target Translation:  es muy depresivo 
Predicted Target Translation:  hay


# Transformer Model  

In [40]:
!pip install transformers shap -q

     |████████████████████████████████| 3.3 MB 12.7 MB/s 
     |████████████████████████████████| 564 kB 37.6 MB/s 
     |████████████████████████████████| 3.3 MB 38.8 MB/s 
     |████████████████████████████████| 596 kB 44.6 MB/s 
     |████████████████████████████████| 61 kB 505 kB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 


In [41]:
!pip install sentencepiece -q

     |████████████████████████████████| 1.2 MB 13.1 MB/s 


In [42]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import shap
import torch

In [45]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es")

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

In [46]:
data = ["Transformers are quickly become the model of choice over NLP problems, because Attention is all you need!"]

In [50]:
explainer = shap.Explainer(model, tokenizer)
shap_values = explainer(['he is very depressed'], fixed_context=1)

In [55]:
print('Input Source sentence:', X_test[k:k+1].values[0])
print('Actual Target Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Target Translation with Transformer:')
# View the i/o heatmap
shap.plots.text(shap_values)

Input Source sentence: he is very depressed
Actual Target Translation:  es muy depresivo 
Predicted Target Translation with Transformer:
